# Mongo-project

## Description
### Working with companies data frame

- Import the thisng that I'll need

In [1]:
from mongofunctions import *
import pandas as pd
import numpy as np
import requests
import re
from pymongo import MongoClient
from pandas.io.json import json_normalize
import json


### Analize what is the most imporatnat caracteristics to acomplish

In [2]:
# Pupulation divided by percentage
employees= [(20,"Designers"),(5, "UI/UX Engineers"), (10, "Frontend Developers"), (15,"Data Engineers"),
            (5,"Backend Developers"), (20, "Account Managers"),(1,"Maintenance guy that loves basketball"),
            (10,"Executives"),(1,"CEO/President")]

perc = {f"{e[1]}: {round((e[0]/(sum([e[0] for e in employees]))*100))}%" for e in employees}
perc

{'Account Managers: 23%',
 'Backend Developers: 6%',
 'CEO/President: 1%',
 'Data Engineers: 17%',
 'Designers: 23%',
 'Executives: 11%',
 'Frontend Developers: 11%',
 'Maintenance guy that loves basketball: 1%',
 'UI/UX Engineers: 6%'}

### Based on this precentages this is the pripority list:
    - Companies more than 10-year-old more than 2km ratio (affects 100% of the employees)
    - Bar to party (100%)
    - School nearby (30%)
    - Airport in the city (23%)
    - Starbucks near by (11%)    

# ====================================

## Clean the data base

- Import the database from MongoDB and test it

In [3]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [4]:
#list(db.companies.find({"name":"Facebook"},{"offices":1}))

In [5]:
all_offices = list(db.companies.find({},{"_id": 0,"name":1,"category_code":1, "offices":1, 
                                       "founded_year":1,"total_money_raised":1}))

In [6]:
companydata = pd.DataFrame(all_offices)

In [7]:
companydata = companydata.explode("offices")

In [8]:
cleaned_offices = companydata.apply(officeToGeoPoint,axis=1, result_type="expand")
cleaned_offices.columns = ["office","clean_state"]

- Get the address of the offices

In [9]:
young_off = list(db.companies.aggregate([{"$unwind":"$offices"},
                                    {"$project":{"name":1,"founded_year":1,"category_code":1,
                                                 "total_money_raised":1,"offices":1}}]))
young_offices = pd.DataFrame(young_off)
y_off = young_offices.apply(officeToAddress,axis=1, result_type="expand")

In [10]:
finall = pd.concat([companydata,cleaned_offices,y_off], axis = 1, sort=False)

In [11]:
finall_offi = finall.rename(columns={"office": "geopoint",
                                    0: "address",
                                    1: "city"})

- Get the coordenates of each office

In [12]:
y_geo = young_offices.apply(officeToLatLong,axis=1, result_type="reduce")
final = pd.concat([finall_offi,y_geo], axis = 1, sort=False)

In [13]:
final_off = final.rename(columns={0: "geocoord"})

- Select just the offices that have a valid data for:
    - Adress
    - Founded year
    - City
    - Geo-coordenates

In [14]:
# comp = complete.drop(complete[complete.founded_year < 2010].index)
comple = final_off.dropna(subset=["founded_year","total_money_raised","geopoint"])
complet = comple.drop_duplicates(subset='geocoord')
print(len(comple))
print(len(complet))

9122
4157



## Select office locations based on the conditions decided before

- Companies that have been founded less than 10 years ago

In [15]:
# Select just the places that have raised at least 1 million and are less than 10 year old
y_offices = complet.loc[complet['founded_year'] > 2010]
print(len(y_offices))

22


- To exclude the old companies I need to create a database of all companies founded more than 10 years ago

In [16]:
# Select just companies fonded before 2010 to compare this database ans exclude locations near by
old_off = complet.loc[complet['founded_year'] <= 2010]
print(len(old_off))

4135


In [17]:
#y_offices.to_json("../input/y_offices.json", orient="records")
#design_off.to_json("../input/design_off.json", orient="records")
old_off.to_json("../input/old_off.json", orient="records")

### Analize compare the dataframe with the employees needs

- Compare the possible locations with the **old companies** database

In [18]:
# I tried to do it with adress, but it was not possible because geocode.xys takes to long and returns many errors
y_address = list(''.join(e) for e in zip(y_offices['address'],y_offices['city']))

In [19]:
# So I decided to do it creating the geopints myself
points = list([e for e in y_offices['geopoint']])
point_list = list([pointNear(p) for p in points])

In [20]:
# I had to create index for the data base and import i tagain -->db.old_off.createIndex( { geometry : "2dsphere" } )
client = MongoClient("mongodb://localhost/old_off")
db = client.get_database()

In [21]:
checkoffclose = set([ckeck2km(db.old_off,e,'old offices') for e in point_list])
checkoffclose

{'there are NO old offices near by'}

- Check if there is a **bar** 500m near, for employees to go party

In [22]:
y_off_bar = checkNear(y_offices,500,"bar")
len(y_off_bar)

This companies do has a bar near by: ['PeekYou', 'headr', 'Fixya', 'RazorGator', 'Skydeck', 'FirstString', 'SEOGroup', 'Advisor', 'Non-Member Films', 'Kidos', 'Clowdy', 'Queralt', 'LoveAccess', 'RipCloud', 'Plug in SEO', 'Shopseen']
This companies does NOT has a bar near by: ['Fuzz', 'Carfeine', 'Ziippi', 'Pixelmatic', 'Cyphercor', 'Fliggo']


16

- Check wich of the locations has a **school** 1KM near

In [23]:
y_off_school = checkNear(y_off_bar,1000,"elementary school")
len(y_off_school)

This companies do has a elementary school near by: ['PeekYou', 'headr', 'Fixya', 'RazorGator', 'Skydeck', 'Advisor', 'Non-Member Films', 'Kidos', 'Clowdy', 'Queralt', 'LoveAccess', 'Plug in SEO', 'Shopseen']
This companies does NOT has a elementary school near by: ['FirstString', 'SEOGroup', 'RipCloud']


13

- Check if there is a **Starbuck** 100 near, because exacutives likes a lot Starbucks coffee

In [24]:
y_off_sands= checkNear(y_off_school,100,"Starbucks")
len(y_off_sands)

This companies do has a Starbucks near by: ['PeekYou', 'headr', 'RazorGator', 'Clowdy', 'LoveAccess', 'Plug in SEO', 'Shopseen']
This companies does NOT has a Starbucks near by: ['Fixya', 'Skydeck', 'Advisor', 'Non-Member Films', 'Kidos', 'Queralt']


7

- Validate if there is an **Airport** in the proposed cities

In [25]:
airports = pd.read_csv('../input/airports2.csv', encoding='latin-1', error_bad_lines=False)
airports[["airport_name", "City", "Country", "airport_code"," ","Latitude", "Longitude"
         ]] = airports["airport_name;City;Country;airport_code;;Latitude;Longitude"].str.split(";",expand=True)

airp = airports.drop(['airport_name;City;Country;airport_code;;Latitude;Longitude', 'airport_code'], axis=1)
airp_cities = set(map(str.strip, list(e for e in airp['City'])))

In [26]:
airp_cities = set(map(str.strip, list(e for e in airp['City'])))
airport = []
noairport = []
for i,e in y_off_sands.iterrows():
    if y_off_sands['city'][i] in airp_cities:
        airport.append(y_off_sands['city'][i])
    else:
        noairport.append(y_off_sands['city'][i])
print(f"These companies are in cities with airport: {airport}")
print(f"These companies are in cities tha DON'T have airport:{noairport}")

These companies are in cities with airport: ['Cambridge', 'New York', 'Washington', 'Chicago', 'Vancouver']
These companies are in cities tha DON'T have airport:['Sausalito', 'Bracknell']


In [27]:
# Dorp locations that does not have a starbucks near
noairp = ['Sausalito', 'Bracknell']
y_airp = y_off_sands[~y_off_sands['city'].isin(noairp)]
len(y_airp)

5

## There are 5 possible places lets represent them in a map

In [28]:
# Clean the data frame
clean = y_airp.drop(columns=['category_code', 'founded_year','total_money_raised','offices','clean_state'])
clean['type']= 'Company'
clean.to_json("../output/clean.json", orient="records")

In [29]:
bares = getPlaces(clean,'bar')
schools = getPlaces(clean,'elementary school')
starbucks = getPlaces(clean,'Starbucks')
aeropuertos = getPlaces(clean,'airport')

In [30]:
bares1 = pd.DataFrame(bares)
bares1['type']= 'Bar'
schools1 = pd.DataFrame(schools)
schools1['type']= 'School'
starbucks1 = pd.DataFrame(starbucks)
starbucks1['type']= 'Starbucks'
aeropuertos1 = pd.DataFrame(aeropuertos)
aeropuertos1['type']= 'Airports'
all_places = pd.concat([bares1,schools1,starbucks1,aeropuertos1], axis = 0, sort=False)
all_places.to_json("../input/all_places.json", orient="records")

# The best places to grow is near to the following locations:

In [31]:
lista = [(clean['address'][i],clean['city'][i]) for i,e in clean.iterrows()]
lista

[('17 Dunster Street', 'Cambridge'),
 ('770 Broadway', 'New York'),
 ('2445 M St NW', 'Washington'),
 ('200 South Wacker Drive', 'Chicago'),
 ('700 - 1112 West Pender St.', 'Vancouver')]

To see the exact location in a map in Tableau click in this [link](https://public.tableau.com/profile/ana8098#!/vizhome/Mongo-project-places/Sheet1?publish=yes)

To see the locations near by in a map, clik in this [link](https://public.tableau.com/profile/ana8098#!/vizhome/Mongo-project-nearby/Sheet2?publish=yes)

--------------------------------------------------

- Rest of conditions:
    - Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
    - Developers like to be near successful tech startups that have raised at least 1 Million dollars.
    - The CEO is Vegan